Hate Speech and Explicit Content Detection

 # <----- Data Pre-processing ----->

dataset: https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset

class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither

In [1]:
# Imports

import pandas as pd
import numpy as np
import random

In [2]:
# Importing dataset

file = "Datasets/Dataset1/archive/labeled_data.csv"

df = pd.read_csv(file)

df.shape

(24783, 7)

In [3]:
df.head(50)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


# <----- Dataset Optimisation ----->

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [5]:
# Missing values

df.isnull().any()

Unnamed: 0            False
count                 False
hate_speech           False
offensive_language    False
neither               False
class                 False
tweet                 False
dtype: bool

In [6]:
# Duplicate Values

duplicated_rows_df = df[df.duplicated()]
print("No of dup rows: ", duplicated_rows_df.shape)

No of dup rows:  (0, 7)


# <----- Dataset Textual Analysis Prep ----->

In [7]:
# More imports

import textacy
import textacy.preprocessing.normalize as tprep
import unidecode
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [8]:
# <-- RUN ONCE -->

#nltk.download('stopwords')
#nltk.download("wordnet")
#nltk.download("onw-1.4")
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\jakeh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\jakeh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\jakeh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\jakeh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\jakeh\AppData\Roaming\nltk_data...
[

True

In [9]:
# Data Normalization and cleaning

df.head(1)

tweet = df['tweet'].to_numpy()

tweet2 = []

for x in tweet:
    x = x.lower()
    x = tprep.hyphenated_words(x)
    x = tprep.quotation_marks(x)
    x = tprep.unicode(x)
    x = unidecode.unidecode(x)
    x = re.sub(r'[^\w\s]', '', x)

    tweet2.append(x)

df_tweet = pd.Series(tweet2)

df_tweet.head(20)


0      rt mayasolovely as a woman you shouldnt compl...
1      rt mleew17 boy dats coldtyga dwn bad for cuff...
2      rt urkindofbrand dawg rt 80sbaby4life you eve...
3      rt c_g_anderson viva_based she look like a tr...
4      rt shenikaroberts the shit you hear about me ...
5     t_madison_x the shit just blows meclaim you so...
6     __brighterdays i can not just sit up and hate ...
7     8220selfiequeenbri cause im tired of you big b...
8      amp you might not get ya bitch back amp thats...
9      rhythmixx_ hobbies include fighting mariam\n\...
10     keeks is a bitch she curves everyone  lol i w...
11                      murda gang bitch its gang land 
12        so hoes that smoke are losers   yea  go on ig
13           bad bitches is the only thing that i like 
14                                 bitch get up off me 
15                         bitch nigga miss me with it 
16                                  bitch plz whatever 
17                               bitch who do yo

In [10]:
# Stopword Removal

stop_words = set(stopwords.words('english'))

df_tweets2 = df_tweet.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

df_tweets2.shape

(24783,)

In [11]:
df_tweets2.head(5)


0    rt mayasolovely woman shouldnt complain cleani...
1    rt mleew17 boy dats coldtyga dwn bad cuffin da...
2    rt urkindofbrand dawg rt 80sbaby4life ever fuc...
3          rt c_g_anderson viva_based look like tranny
4    rt shenikaroberts shit hear might true might f...
dtype: object

In [12]:
# Lemmatization

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

lemm_data = df_tweets2.to_numpy()

for x in lemm_data:
    str = x.split()
    for x in str:
        x = lemmatizer.lemmatize(x)

df_tweets3 = pd.Series(lemm_data)

df_tweets3.head(10)


0    rt mayasolovely woman shouldnt complain cleani...
1    rt mleew17 boy dats coldtyga dwn bad cuffin da...
2    rt urkindofbrand dawg rt 80sbaby4life ever fuc...
3          rt c_g_anderson viva_based look like tranny
4    rt shenikaroberts shit hear might true might f...
5    t_madison_x shit blows meclaim faithful somebo...
6    __brighterdays sit hate another bitch got much...
7    8220selfiequeenbri cause im tired big bitches ...
8                amp might get ya bitch back amp thats
9     rhythmixx_ hobbies include fighting mariam bitch
dtype: object

# <----- Teaxtaul Analysis ----->

In [13]:
# Imports

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
# Dataset Values

final_tweets = df_tweets3
value =  df['class']

processed_df = pd.DataFrame({'tweets': final_tweets, 'value': value})

processed_df.head(78)


,tweets,value
0,rt mayasolovely woman shouldnt complain cleani...,2
1,rt mleew17 boy dats coldtyga dwn bad cuffin da...,1
2,rt urkindofbrand dawg rt 80sbaby4life ever fuc...,1
3,rt c_g_anderson viva_based look like tranny,1
4,rt shenikaroberts shit hear might true might f...,1
...,...,...
73,bro_hen314 eaglesnation every eagles need see ...,1
74,bvsedchink yo fuck skateboarding yall wood pus...,1
75,babyanimalpics baby monkey bathtime httptco7kp...,2
76,baylaagottabody 128514128514128514128514 aint ...,1


In [15]:
# Sentiment Analysis

analyser = SentimentIntensityAnalyzer()

def getSentiment(text):
    scores = analyser.polarity_scores(text)
    sentiment = 1 if scores['pos'] > 0 else 0

    return sentiment

processed_df['sentiment'] = processed_df['tweets'].apply(getSentiment)

sentiment = processed_df['sentiment'].to_numpy()

processed_df.head(10)

,tweets,value,sentiment
0,rt mayasolovely woman shouldnt complain cleani...,2,1
1,rt mleew17 boy dats coldtyga dwn bad cuffin da...,1,0
2,rt urkindofbrand dawg rt 80sbaby4life ever fuc...,1,0
3,rt c_g_anderson viva_based look like tranny,1,1
4,rt shenikaroberts shit hear might true might f...,1,1
5,t_madison_x shit blows meclaim faithful somebo...,1,1
6,__brighterdays sit hate another bitch got much...,1,0
7,8220selfiequeenbri cause im tired big bitches ...,1,0
8,amp might get ya bitch back amp thats,1,0
9,rhythmixx_ hobbies include fighting mariam bitch,1,0


In [16]:
# TF-IDF Score

the_text = processed_df['tweets']

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None, max_features=20)

tfidf_tweets = tfidf.fit_transform(the_text)

vocab = tfidf.vocabulary_
feature_names = list(vocab.keys())

df_tfidf = pd.DataFrame(tfidf_tweets.toarray(), columns=feature_names)

df_tfidf.head(1)

,rt,amp,trash,hoe,fuck,bitch,shit,like,hoes,got,im,bitches,get,lol,nigga,dont,pussy,aint,ass,know
0,0.0,0.691184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.345537,0.0,0.63472


In [ ]:
# Bag of words



In [17]:
# final Dataset

df_final = df_tfidf

df_final['orig_tweets'] = final_tweets
df_final['sentiment_value'] = sentiment
df_final['class_value'] = value

df_final.shape

(24783, 23)

In [18]:
# Export to csv file

df_final.to_csv('dataset1Finalisedcol23.csv', index=False)